In [1]:
import numpy as np 
import pandas as pd 

import nltk
import json

from gensim.models.doc2vec import Doc2Vec, TaggedDocument


In [2]:
target = pd.read_csv('/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/target.csv')
print(len(target))
target.head()

8703


,id,date,2,3,5,6,7,8,9,10,...,19,34,35,37,41,46,P1,P4,P12,P7
0,HUDOC-ECHR-1999-001-58225,1999-03-25,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,2,-1,-1,-1,-1,-1
1,HUDOC-ECHR-1999-001-58226,1999-03-25,-1,-1,-1,0,-1,-1,-1,-1,...,-1,-1,-1,-1,2,-1,-1,-1,-1,-1
2,HUDOC-ECHR-1999-001-58227,1999-03-25,-1,-1,-1,2,-1,2,-1,-1,...,-1,-1,2,-1,2,-1,2,-1,-1,-1
3,HUDOC-ECHR-1999-001-58239,1999-04-29,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,2,-1,2,-1,-1,-1,-1,-1
4,HUDOC-ECHR-1999-001-58251,1999-05-20,0,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,2,-1,2,-1,-1,-1,-1,-1


In [3]:
#Open cases
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/{}.json"

with open(path.format('text_alpha'), 'r') as readfile:
    text = json.load(readfile)
    readfile.close()
    
print(len(text.keys()))
print(list(text.keys())[0])
print(len(text['HUDOC-ECHR-2012-001-110881']))

8703
HUDOC-ECHR-1999-001-58225
7


In [18]:
def featureSet(article,part,model):
    """
    returns the feature set corrisponding to article and part 
    """
    df = target[['id',article]]
    
    #Get balanced dataset
    nvLen = len(df[df[article] == 0]) 
    vLen = len(df[df[article] == 1])
    minLen = min(nvLen,vLen)

    nvID = df[df[article] == 0][0:minLen]['id']
    vID = df[df[article] == 1][0:minLen]['id']
    
    nvCorpus = []
    vCorpus = []
    for ID in nvID:
        nvCorpus.append(text[ID][part])

    for ID in vID:
        vCorpus.append(text[ID][part])
        
    corpus = nvCorpus + vCorpus
    targets = ['nonviolation']*minLen + ['violation']*minLen
    
    
    docs_vectors = pd.DataFrame() # creating empty final dataframe
    for doc in corpus:
       
        doc_vector = pd.Series(model.infer_vector(doc,steps = 20))
        
        docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
    
    docs_vectors['target'] = targets
        
    return docs_vectors    

In [5]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/embeddings/{}"
model = Doc2Vec.load(path.format('doc2vec2_100')) 

In [25]:
article = '6'
part = 'procedure'
model = Doc2Vec.load(path.format('doc2vec2_100')) 
print("Model Loaded")
features = featureSet(article,part,model)
features.head()

/Users/conorosully/miniconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Model Loaded


,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,target
0,-0.516066,2.107780,2.162260,2.616603,-1.863217,0.816716,-4.916944,-4.234544,-1.074629,-0.285121,...,0.059982,1.866904,0.643663,-1.957397,-0.911621,1.269738,-1.552932,-1.261980,-0.779531,nonviolation
1,-0.073905,1.793918,2.443634,2.896621,-1.538860,0.433552,-5.224380,-4.631708,-0.863127,0.012287,...,0.001588,2.381957,0.259254,-2.059462,-1.101341,1.332176,-1.531820,-0.825848,-1.178080,nonviolation
2,0.017941,1.818290,2.521929,3.262511,-1.434001,0.895707,-4.929165,-4.615604,-0.888794,-0.488479,...,-0.240790,2.267651,0.577258,-2.407805,-1.156552,1.300444,-1.619178,-1.127879,-1.090577,nonviolation
3,-0.335421,1.886686,2.761087,3.460682,-1.106604,1.154939,-5.206748,-4.722277,-1.216418,-0.769439,...,-0.465992,2.309606,0.208398,-1.949610,-1.264699,1.215914,-1.431834,-0.876558,-1.002460,nonviolation
4,-0.538657,1.704808,2.352867,2.342612,-0.759345,1.082458,-4.337107,-3.670284,-1.111958,-0.189889,...,-0.157212,1.478003,0.578263,-1.961035,-1.159985,1.057021,-1.278481,-0.911024,-0.901303,nonviolation


## Fit Models

In [12]:
#Imports
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#import autosklearn.classification

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import json

import functions 
classif = functions.classification()

In [13]:
def trainTest(features):
    """Return train test split from features"""
    #print(features['target'])
    X = features.drop(columns='target')
    X=(X-X.mean())/X.std() #Normalise
    y = features['target']
    return train_test_split(X, y, test_size=0.10, random_state=90)

def k_fold(X_train, y_train,C,cv = 10,kernel='linear'):
    """
    Returns k_fold accuracy for given C parameter
    """
    model = SVC(kernel=kernel, C=C)
    scores = cross_val_score(model, X_train, y_train, cv=cv)
    accuracy = scores.mean()
    return accuracy

In [26]:
article = '6'
part = 'full'
model = Doc2Vec.load(path.format('doc2vec2_100')) 
features = featureSet(article,part,model)
X_train, X_test, y_train, y_test = trainTest(features)
print("Model Loaded")

hyper_c = [0.0001,0.001,0.01,0.1,1,10]
for C in hyper_c:
    accuracy = k_fold(X_train, y_train,C)
    print(C, accuracy)

/Users/conorosully/miniconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Model Loaded
0.0001 0.6865325801852397
0.001 0.6853281236962302
0.01 0.6949516222069851
0.1 0.6804342936334317
1 0.675615016524998
10 0.6646533741107157


In [27]:
article = '6'
part = 'full'
model = Doc2Vec.load(path.format('doc2vec2_200')) 
features = featureSet(article,part,model)
X_train, X_test, y_train, y_test = trainTest(features)
print("Model Loaded")

hyper_c = [0.0001,0.001,0.01,0.1,1,10]
for C in hyper_c:
    accuracy = k_fold(X_train, y_train,C)
    print(C, accuracy)

/Users/conorosully/miniconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Model Loaded
0.0001 0.6962306316503593
0.001 0.6913371643756598
0.01 0.684123667207221
0.1 0.6525765210798025
1 0.6463739329495035
10 0.6293881941496792


In [ ]:
##### article = '6'
part = 'full'
model = Doc2Vec.load(path.format('doc2vec2_300')) 
features = featureSet(article,part,model)
X_train, X_test, y_train, y_test = trainTest(features)
print("Model Loaded")

hyper_c = [0.0001,0.001,0.01,0.1,1,10]
for C in hyper_c:
    accuracy = k_fold(X_train, y_train,C)
    print(C, accuracy)